In [1]:
"""
Baseline fact-checking implementation using LLM's internal knowledge only.

This script implements the baseline approach where the LLM makes fact-checking
decisions based solely on its training data, without access to external tools
or real-time information retrieval. This serves as the control group for
measuring the effectiveness of tool-augmented reasoning.

The baseline approach:
1. Loads claims from the processed binary dataset
2. Uses simple prompting to ask the LLM for True/False judgments
3. Records predictions and justifications
4. Saves results for later comparison with tool-augmented approach
"""

"\nBaseline fact-checking implementation using LLM's internal knowledge only.\n\nThis script implements the baseline approach where the LLM makes fact-checking\ndecisions based solely on its training data, without access to external tools\nor real-time information retrieval. This serves as the control group for\nmeasuring the effectiveness of tool-augmented reasoning.\n\nThe baseline approach:\n1. Loads claims from the processed binary dataset\n2. Uses simple prompting to ask the LLM for True/False judgments\n3. Records predictions and justifications\n4. Saves results for later comparison with tool-augmented approach\n"

In [2]:
!pip install -U pandas tqdm openai langchain langchain_community wikipedia requests


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 8.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 109.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 767.8/767.8 kB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 105.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.8/64.8 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.2 MB/s eta 0:00:00
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11678 sha256=77c810a2c14154c2c009edf0c109d43a1a6d05c760b2cba800911f75ab9ba9be
  Stored in directory: /root/.cache/pip/wheels/8f/ab/cb/45ccc40522d3a1c41e1d2ad53b8f33a62f394011ec38cd71c6
Successfully built wikipedia
  Attempting uninstall: requests
    Found existing installation: requests 2.32.3
    Uninstalling requests-2.32.3

In [3]:
# Don't forget!
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
import sys
import os
from typing import List, Dict
import logging
from tqdm import tqdm

In [5]:
os.chdir('/content/drive/MyDrive/DS301_Final_Project')

In [6]:
from src.utils import simple_llm_call, parse_llm_response, load_prompt, save_results

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
# Clear existing handlers and force reconfigure
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

logging.basicConfig(level=logging.INFO,
                    format='%(levelname)s:%(name)s:%(message)s',
                    handlers=[
                        logging.StreamHandler(sys.stdout)
                    ],
                    force=True)
logger = logging.getLogger(__name__)

In [8]:
class BaselineFactChecker:
  """
   Baseline fact-checker that relies only on LLM's internal knowledge.

   This class implements the simplest fact-checking approach:
   present the claim to the LLM and ask for a True/False judgment.
   No external evidence retrieval or tool usage is involved.

   This serves as our control group to measure how much improvement
   we get from adding tool-augmented reasoning.
   """
  def __init__(self, model_name: str = "gpt-3.5-turbo", prompt_file: str = "baseline_prompt.txt"):
    """
       Initialize the baseline fact-checker.

       Args:
           model_name: Name of the LLM model to use (we use gpt-3.5-turbo)
           prompt_file: Prompt template file in the prompts/ directory
       """
    self.model_name = model_name
    # Load the baseline prompt template that will guide the LLM's responses
    try:
      self.prompt_template = load_prompt(prompt_file)
      logger.info(f"Loaded prompt template from {prompt_file}")
    except FileNotFoundError:
      logger.error(f"Prompt file {prompt_file} not found in prompts/ directory")
      raise

    logger.info(f"Initialized baseline fact-checker with {model_name}")

  def check_single_claim(self, claim: str, claim_id: str = None) -> Dict[str, str]:
    """
      Fact-check a single claim using only the LLM's internal knowledge.

      This is the core baseline method that:
      1. Takes a claim as input
      2. Formats it using the baseline prompt template
      3. Sends it to the LLM with zero-shot prompting
      4. Parses the response for True/False prediction and justification

      No external evidence is retrieved - the LLM must rely entirely on
      its training data, which may be outdated or incomplete.

      Args:
          claim: The factual statement to verify
          claim_id: Optional identifier for tracking this claim

      Returns:
          Dictionary containing prediction, justification, and metadata
      """
    # Insert the specific claim into our prompt template
    formatted_prompt = self.prompt_template.format(claim=claim) #

    # Make a simple call to the LLM without any system message
    # This is pure zero-shot prompting - no additional context provided
    raw_response = simple_llm_call(
          prompt=formatted_prompt,
          model=self.model_name
    )

    # Parse the LLM's response to extract structured prediction and reasoning
    parsed_result = parse_llm_response(raw_response)

    # Package the results with metadata for analysis
    result = {
      'claim_id': claim_id,
      'claim': claim,
      'prediction': parsed_result['prediction'],  # True/False/Unknown
      'justification': parsed_result['justification'],  # LLM's reasoning
      'raw_response': raw_response,  # Full LLM output for debugging
      'method': 'baseline',  # Tag to distinguish from tool-augmented results
      'model': self.model_name
    }

    return result

  def check_multiple_claims(self, claims_df: pd.DataFrame, max_samples: int = None) -> List[Dict]:
    """
    Fact-check multiple claims from a dataset with progress tracking.

    This method processes claims in batch, handling errors gracefully
    and providing progress updates. It's designed to process the entire
    test set while being robust to individual claim failures.

    Args:
        claims_df: DataFrame with 'id', 'statement', and 'binary_label' columns
        max_samples: Optional limit on number of claims to process (for testing)

    Returns:
        List of result dictionaries, one per claim processed
    """
    results = []

    # Limit sample size if specified (useful for testing)
    if max_samples:
      claims_df = claims_df.head(max_samples)
      logger.info(f"Processing first {max_samples} claims")

    logger.info(f"Starting baseline fact-checking on {len(claims_df)} claims")

    # Process each claim with a progress bar for user feedback
    for idx, row in tqdm(claims_df.iterrows(),
                        # iterrows enables iterate through pd df by row,
                        # by returning (idx, row)
                          total=len(claims_df),
                          desc="Baseline fact-checking"): # info aid for bar
      try:
        # Attempt to fact-check this individual claim
        result = self.check_single_claim(
            claim=row['statement'],
            claim_id=row.get('id', str(idx))
        )
        # Add the ground truth label for later evaluation
        result['ground_truth'] = row['binary_label']
        results.append(result)

        # Log progress every 10 claims
        if (idx + 1) % 10 == 0:
          successful = len([r for r in results if r['prediction'] != 'Error'])
          logger.info(f'Processed {idx + 1} claims, {successful} successful')

      except Exception as e:
        # Log the error but continue processing other claims
        logger.error(f"Error processing claim {idx}: {e}")
        # Create an error result to maintain dataset alignment
        # This ensures we have a result for every input claim
        error_result = {
          'claim_id': row.get('id', str(idx)),
          'claim': row['statement'],
          'prediction': 'Error',
          'justification': f'Processing failed: {str(e)}',
          'ground_truth': row['binary_label'],
          'raw_response': f'Error: {str(e)}',
          'method': 'baseline',
          'model': self.model_name
        }
        results.append(error_result)

    logger.info(f"Completed baseline fact-checking. Processed {len(results)} claims")
    return results




In [9]:
def run_baseline_experiment(
   train_data_path: str = "/content/drive/MyDrive/DS301_Final_Project/data/train_binary.csv",
   test_data_path: str = "/content/drive/MyDrive/DS301_Final_Project/data/test_binary.csv",
   output_path: str = "/content/drive/MyDrive/DS301_Final_Project/results/baseline_preds.jsonl",
   max_samples: int = None,
   model_name: str = "gpt-3.5-turbo"
):
  """
  Run the complete baseline fact-checking experiment.

  This function orchestrates the entire baseline experiment workflow:
  1. Loads the official LIAR dataset train/test split
  2. Runs baseline fact-checking on the test set
  3. Saves detailed results for later evaluation and comparison

  We use the official test set to ensure our results are comparable
  to other research on the LIAR dataset.

  Args:
      train_data_path: Path to the processed training data (for reference)
      test_data_path: Path to the processed test data (what we evaluate on)
      output_path: Where to save the prediction results
      max_samples: Optional limit on test samples (useful for debugging)
      model_name: LLM model to use for fact-checking
  """
  logger.info("="*60)
  logger.info("STARTING BASELINE FACT-CHECKING EXPERIMENT")
  logger.info("="*60)

  # Load the official LIAR dataset splits
  logger.info("Loading LIAR dataset splits...")

  try:
      # Load training data (just for statistics and reference)
      train_df = pd.read_csv(train_data_path)
      logger.info(f"Loaded training data: {len(train_df)} samples")

      # Load test data (this is what we'll actually evaluate on)
      test_df = pd.read_csv(test_data_path)
      logger.info(f"Loaded test data: {len(test_df)} samples")
  except FileNotFoundError as e:
      logger.error(f"Dataset file not found: {e}")
      logger.info("Please ensure both train_binary.csv and test_binary.csv exist in your data/ folder")
      return

  # Display dataset statistics for context
  logger.info("\nDataset Statistics:")
  logger.info(f"Training set label distribution:")
  logger.info(train_df['binary_label'].value_counts())
  logger.info(f"Test set label distribution:")
  logger.info(test_df['binary_label'].value_counts())

  # Initialize the baseline fact-checker
  logger.info(f"Initializing the baseline fact-checker with {model_name}...")
  try:
    fact_checker = BaselineFactChecker(model_name=model_name)
  except Exception as e:
    logger.error(f"Failed to initialize fact-checker: {e}")

  # Run fact-checking on the test set
  logger.info("Starting fact-checking process...")

  results = fact_checker.check_multiple_claims(test_df, max_samples=max_samples)


  # Save results for analysis
  logger.info(f"Saving results to {output_path}")
  save_results(results, output_path)


  # Calculate and display summary statistics
  logger.info("="*60)
  logger.info("BASELINE EXPERIMENT SUMMARY")
  logger.info("="*60)

  successful_preds = [r for r in results if r['prediction'] != 'Error']
  error_count = len(results) - len(successful_preds)

  logger.info(f"Total claims processed: {len(results)}")
  logger.info(f"Successful predictions: {len(successful_preds)}")
  logger.info(f"Failed predictions: {error_count}")

  if successful_preds: # if nonempty
    # Analyze prediction distribution
    pred_counts = {}
    for result in successful_preds:
      pred = result['prediction'] # a str
      pred_counts[pred] = pred_counts.get(pred, 0) + 1 # increment value

    logger.info(f"Prediction distribution: {pred_counts}")
    # Basic accuracy calculation
    correct_predictions = sum(1 for r in successful_preds
                          if str(r['prediction']).lower() == str(r['ground_truth']).lower())
    accuracy = correct_predictions / len(successful_preds) if successful_preds else 0
    accuracy_info=f"\nBasic accuracy: {accuracy:.3f} ({correct_predictions}/{len(successful_preds)})"
    logger.info(accuracy_info)

  logger.info(f"\nResults saved to: {output_path}")
  logger.info("Next steps:")
  logger.info("1. Run tool-augmented experiment: python src/run_tool_augmented.py")
  logger.info("2. Compare results: python src/evaluate.py")

  return results


In [10]:
!pip install python-dotenv

In [13]:
from dotenv import load_dotenv
env_path = "/content/drive/MyDrive/env.txt"
load_dotenv(dotenv_path=env_path)

results = run_baseline_experiment(max_samples=300)

print(f"\nBaseline experiment completed! Processed {len(results)} claims.")

INFO:__main__:============================================================
INFO:__main__:STARTING BASELINE FACT-CHECKING EXPERIMENT
INFO:__main__:============================================================
INFO:__main__:Loading LIAR dataset splits...
INFO:__main__:Loaded training data: 6569 samples
INFO:__main__:Loaded test data: 810 samples
INFO:__main__:
Dataset Statistics:
INFO:__main__:Training set label distribution:
INFO:__main__:binary_label
True     4076
False    2493
Name: count, dtype: int64
INFO:__main__:Test set label distribution:
INFO:__main__:binary_label
True     506
False    304
Name: count, dtype: int64
INFO:__main__:Initializing the baseline fact-checker with gpt-3.5-turbo...
INFO:__main__:Loaded prompt template from baseline_prompt.txt
INFO:__main__:Initialized baseline fact-checker with gpt-3.5-turbo
INFO:__main__:Starting fact-checking process...
INFO:__main__:Processing first 300 claims
INFO:__main__:Starting baseline fact-checking on 300 claims


Baseline fact-checking:   0%|          | 0/300 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Baseline fact-checking:   0%|          | 1/300 [00:00<04:45,  1.05it/s]

DEBUG: Parsing response: 'Answer: False\n\nJustification: There is no specific evidence or information provided to support the claim that Suzanne Bonamici supports a plan that will cut choice for Medicare Advantage seniors. With'
DEBUG: Found prediction: False

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


...

DEBUG: Parsing response: 'Answer: FALSE\nJustification: The claim is false. In the United States, the corporate tax rate is generally higher than the tax rate for small businesses.'
DEBUG: Found prediction: False

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Baseline fact-checking: 100%|█████████▉| 299/300 [05:22<00:01,  1.22s/it]

DEBUG: Parsing response: 'Answer: TRUE\nJustification: In an interview with CNBC in 2016, Donald Trump did suggest that he would try to negotiate down the national debt, stating "I would borrow, knowing that if the economy cras'
DEBUG: Found prediction: True

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
DEBUG: Parsing response: 'Answer: FALSE\nJustification: The claim is false because it is not providing accurate information. While it is true that Donald Trump received a significant amount of free media coverage during the 201'
DEBUG: Found prediction: False

INFO:__main__:Processed 300 claims, 300 successful


Baseline fact-checking: 100%|██████████| 300/300 [05:23<00:00,  1.08s/it]

INFO:__main__:Completed baseline fact-checking. Processed 300 claims
INFO:__main__:Saving results to /content/drive/MyDrive/DS301_Final_Project/results/baseline_preds.jsonl
INFO:__main__:============================================================
INFO:__main__:BASELINE EXPERIMENT SUMMARY
INFO:__main__:============================================================
INFO:__main__:Total claims processed: 300
INFO:__main__:Successful predictions: 300
INFO:__main__:Failed predictions: 0
INFO:__main__:Prediction distribution: {'False': 257, 'True': 43}
INFO:__main__:
Basic accuracy: 0.473 (142/300)
INFO:__main__:
Results saved to: /content/drive/MyDrive/DS301_Final_Project/results/baseline_preds.jsonl
INFO:__main__:Next steps:
INFO:__main__:1. Run tool-augmented experiment: python src/run_tool_augmented.py
INFO:__main__:2. Compare results: python src/evaluate.py

Baseline experiment completed! Processed 300 claims.
